In [1]:
# %%capture
# !pip install openai
# !pip install python-dotenv
# !pip install urllib3

In [1]:
import json
import os
from dotenv import load_dotenv
from openai import OpenAI
import time
import function_call_limit_functions as FC
from function_call_tools import tools, instruction1, instruction2
load_dotenv()

True

#### The class AssistantManager will store functions for creating assistants, creating threads for the assistants, adding message to the threads, running the assistants. Then we will wait for the the completions and submit the response back to the assistant again for generating a response.

In [2]:
class AssistantManager:
  def __init__(self,api_key:str,model:str="gpt-3.5-turbo-1106"):
    self.client = OpenAI(api_key=api_key)
    self.model = model
    self.assistant = None
    self.thread = None
    self.run = None
    self.file = None
    self.message = None

  def create_assistant(self,name, instructions, tools,id):
        assistant_file_path = './assistant.json'
        if os.path.exists(assistant_file_path):
          with open(assistant_file_path, 'r') as file:
            assistant_data = json.load(file)
            if f'{id}' in assistant_data:
              assistant_id = assistant_data[f'{id}']['assistant_id']
              self.assistant = self.client.beta.assistants.retrieve(assistant_id)
              file.close()
            else:
               self.assistant = self.client.beta.assistants.create(
                name = name,
                instructions = instructions,
                tools = tools,
                model = self.model
                )
               assistant_data[f'{id}'] = {'assistant_id': self.assistant.id}
               with open(assistant_file_path, 'w') as file:
                json.dump(assistant_data, file)
                file.close()
               
        else:
            self.assistant = self.client.beta.assistants.create(
                name = name,
                instructions = instructions,
                tools = tools,
                model = self.model
            )
            with open(assistant_file_path, 'w') as file:
              json.dump({f'{id}':{'assistant_id': self.assistant.id}}, file)
              print("Created a new assistant and saved the ID.")
              file.close()

  def create_thread(self,id):
      thread_file_path = './assistant_thread.json'
      if os.path.exists(thread_file_path):
        with open(thread_file_path, 'r') as file:
          thread_data = json.load(file)
          if f'{id}' in thread_data:
            thread_id = thread_data[f'{id}']['thread_id']
            self.thread = self.client.beta.threads.retrieve(thread_id = thread_id)
            file.close()
          else:
             self.thread = self.client.beta.threads.create()
             thread_data[f'{id}'] = {'thread_id': self.thread.id}
             with open(thread_file_path, 'w') as file:
              json.dump(thread_data, file)
              file.close()
      else:
        self.thread = self.client.beta.threads.create()
        with open(thread_file_path, 'w') as file:
              json.dump({f'{id}':{'thread_id': self.thread.id}}, file)
              print("Created a new thread and saved the ID.")
              file.close()

  def add_message_to_thread(self,role,content):
    self.message = self.client.beta.threads.messages.create(
      thread_id = self.thread.id,
      role = role,
      content = content
    )

  def run_assistant(self,instructions):
    self.run = self.client.beta.threads.runs.create(
      thread_id = self.thread.id,
      assistant_id=self.assistant.id,
      instructions = instructions
    )

  def process_messages(self):
    messages = self.client.beta.threads.messages.list(thread_id = self.thread.id)

    msg = messages.data[0]
    role = msg.role
    content = msg.content[0].text.value
    #print(f"{role.capitalize}:{content}")
    return content

  def wait_for_completion(self):
    while True:
      time.sleep(5)
      run_status = self.client.beta.threads.runs.retrieve(
        thread_id = self.thread.id,
        run_id = self.run.id
      )
      
      print(run_status.status)
      if run_status.status == 'completed':
        return(self.process_messages())
        break
      elif run_status.status == 'requires_action':
        print("Function Calling ...")
        self.call_required_functions(run_status.required_action.submit_tool_outputs.model_dump())
      elif run_status.status=="failed":
        print(run_status.last_error)
        return ("The assistant got rate limited, try again after some time or refresh the page and try again")
        break
      else:
        print("Waiting for the Assistant to process..") 
  
  def call_required_functions(self, required_actions):
    tool_output = []

    for action in required_actions["tool_calls"]:
      func_name = action['function']['name']
      arguments = json.loads(action['function']['arguments'])
      print(arguments)
    
      func_name = eval("FC."+func_name)
      output = func_name(**arguments)
      tool_output.append(
        {
          "tool_call_id":action['id'],
          "output": str(output)
        }
      )
    
    print("Submitting outputs back to the Assistants...")
    self.client.beta.threads.runs.submit_tool_outputs(
      thread_id = self.thread.id,
      run_id = self.run.id,
      tool_outputs=tool_output
    )

In [3]:
api_key = os.getenv("Open_AI_key")
manager = AssistantManager(api_key)

manager.create_assistant(
    name = "Function_limit",
    instructions=instruction1,
    tools = tools,
    id = "673224"
)

Created a new assistant and saved the ID.


In [10]:
time.sleep(5)
manager.create_thread(id = "673224")

manager.add_message_to_thread(role="user",content="sentiment analysis for the text 'You are stupid'")

manager.run_assistant(instruction2)
print(manager.wait_for_completion())

requires_action
Function Calling ...
{'text': 'You are stupid'}
Submitting outputs back to the Assistants...
completed
The sentiment of the text "You are stupid" is negative. If you need further assistance or have any other requests, feel free to let me know.


In [11]:
time.sleep(5)
manager.create_thread(id = "673224")

manager.add_message_to_thread(role="user",content="check the email rishavmitra3")

manager.run_assistant(instruction2)
print(manager.wait_for_completion())

requires_action
Function Calling ...
{'email': 'rishavmitra3'}
Submitting outputs back to the Assistants...
completed
The email pattern "rishavmitra3" does not match the standard email format. A valid email should look like "example@example.com". If you have a different email you'd like to check, please provide it.


In [13]:
time.sleep(5)
manager.create_thread(id = "673224")

manager.add_message_to_thread(role="user",content="check the password rishavmitra3")

manager.run_assistant(instruction2)
print(manager.wait_for_completion())

requires_action
Function Calling ...
{'password': 'rishavmitra3'}
Submitting outputs back to the Assistants...
completed
The password "rishavmitra3" has been identified as strong. If you have any other requests or need further assistance, feel free to let me know!


In [14]:
time.sleep(5)
manager.create_thread(id = "673224")

manager.add_message_to_thread(role="user",content="calculate the entropy for 30.12 Joules and 100.482 Kelvin")

manager.run_assistant(instruction2)
print(manager.wait_for_completion())

requires_action
Function Calling ...
{'temperature': 100.482, 'delta_energy': 30.12}
Submitting outputs back to the Assistants...
completed
The entropy calculated for the given temperature of 100.482 Kelvin and delta energy of 30.12 Joules is approximately 0.299 Joules per Kelvin.


In [15]:
time.sleep(5)
manager.create_thread(id = "673224")

manager.add_message_to_thread(role="user",content="120 meters and 50 Hz")

manager.run_assistant(instruction2)
print(manager.wait_for_completion())

requires_action
Function Calling ...
{'wavelength': 120, 'frequency': 50}
Submitting outputs back to the Assistants...
completed
The wave speed calculated using a wavelength of 120 meters and a frequency of 50 Hz is 6000 meters per second (m/s).


In [16]:
time.sleep(5)
manager.create_thread(id = "673224")

manager.add_message_to_thread(role="user",content="Calculate the projectile motion for 100degrees and 30 m/s")

manager.run_assistant(instruction2)
print(manager.wait_for_completion())

requires_action
Function Calling ...
{'v0': 30, 'theta': 100}
Submitting outputs back to the Assistants...
completed
The horizontal range of the projectile motion, given an initial velocity of 30 m/s and a launch angle of 100 degrees, is approximately -31.41 meters. Please note that the negative value suggests that the projectile falls short of its starting point. If you have any other questions or need further assistance, feel free to ask!


In [18]:
time.sleep(5)
manager.create_thread(id = "673224")

manager.add_message_to_thread(role="user",content="Convert 32C")

manager.run_assistant(instruction2)
print(manager.wait_for_completion())

requires_action
Function Calling ...
{'celsius': 32}
Submitting outputs back to the Assistants...
completed
32 degrees Celsius is equivalent to 89.6 degrees Fahrenheit.


In [19]:
time.sleep(5)
manager.create_thread(id = "673224")

manager.add_message_to_thread(role="user",content="Find all even square number till 50")

manager.run_assistant(instruction2)
print(manager.wait_for_completion())

requires_action
Function Calling ...
{'n': 50}
Submitting outputs back to the Assistants...
completed
Here are the even squares up to the limit of 50: [0, 4, 16, 36, 64, 100, 144, 196, 256, 324, 400, 484, 576, 676, 784, 900, 1024, 1156, 1296, 1444, 1600, 1764, 1936, 2116, 2304]. If you have any other requests or need further assistance, feel free to ask!


In [21]:
time.sleep(5)
manager.create_thread(id = "673224")

manager.add_message_to_thread(role="user",content="10001101 to decimal")

manager.run_assistant(instruction2)
print(manager.wait_for_completion())

requires_action
Function Calling ...
{'binary': '10001101'}
Submitting outputs back to the Assistants...
completed
The binary number "10001101" is equivalent to the decimal number 141.
